# Introduction
In notebook '12. Manual curation mass balances.ipynb' I fixed the mass balance of many of the reactions assigned to me. After having a meeting with Ben and Martyn, I've gained some input on how to fix the remaining mass imbalanced reactions from the list assigned to me. This notebook will cover fixing those with the approximate ways that was recommended.

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from cobra import Model, Reaction, Metabolite

In [135]:
#e. coli model for comparison where necessary
model_e_coli = cameo.load_model("iML1515")
model_e_coli_MG1655 = cameo.load_model("iJO1366")

In [298]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

In [299]:
model.metabolites.ahdt_c.formula = 'C9H12N5O13P3'
model.metabolites.ahdt_c.charge = -4

In [300]:
model.reactions.AHETDYTTPHY.add_metabolites({model.metabolites.h_c:-1})

In [301]:
model.metabolites.arbt6p_c.name = 'Arbutin 6-phosphate'

In [302]:
model.reactions.ARBTPT.notes['NOTES']:'Phosphate group donor mimicked here'

In [303]:
model.reactions.ARBTPT.add_metabolites({model.metabolites.pi_c:-1, model.metabolites.h2o_c: 1})

In [304]:
model.metabolites.arbt6p_c.formula = 'C12H15O10P'
model.metabolites.arbt6p_c.charge = -2

In [305]:
model.metabolites.glutrna_c.formula = 'C5H7NO3R'

In [306]:
#need to add the tRNA metabolite
model.add_metabolites(Metabolite(id = 'trnaglu_c'))

In [307]:
model.metabolites.trnaglu_c.name = 'TRNA (Glu)'
model.metabolites.trnaglu_c.formula = 'R'
model.metabolites.trnaglu_c.charge = 0
model.metabolites.trnaglu_c.compartment = 'c'

In [308]:
model.metabolites.trnaglu_c.annotation = model_e_coli.metabolites.trnaglu_c.annotation

In [309]:
model.reactions.GLUTRS.add_metabolites({model.metabolites.trnaglu_c:-1})

In [310]:
model.reactions.GLUTRR.add_metabolites({model.metabolites.trnaglu_c:1})

In [311]:
model.metabolites.nh4_c.name = 'Ammonium'
model.metabolites.nh4_e.name = 'Ammonium'

To tackle the reactions involving cellulose, we need to find a new way to represent its hydrolysis to ensure mass balance and correct functioning of the model.

cellulose_c is involved in two reactions: BGLUCH and CELLOHYDRO. The difference between these reactions:
- BGLUCH is the cellulose hydrolysis to beta-D-glucose + cellulose
- CELLOHYDRO is the hydrolysis of cellulose to cellobiose + cellulose.

To deal with this, I will represent the cellulose as two single units, instead of a long chain polymer. This will maintain mass balance and functioning of the model, but is something that should be considered when trying to model growth of the organism on cellulose. 

In [312]:
model.metabolites.cellb_c.name = 'Cellobiose'

In [313]:
model.metabolites.cellulose_c.formula = 'C12H20O10'

In [314]:
model.reactions.CELLOHYDRO.add_metabolites({model.metabolites.cellulose_c:-11})

In [315]:
model.reactions.BGLUCH.add_metabolites({model.metabolites.cellulose_c:10, model.metabolites.glc__bD_c:1, model.metabolites.h2o_c:-1})

For RCT UAAAGGLAAT I will change the chemical formula of gmagg_c to just reflect the group it donates to the reaction (e.g. similar to cellulose) instead of it being a polymer gmagg_c is only in this reaction so it should be oke.


In [316]:
model.metabolites.gmagg_c.formula = 'C39H64N8O19'
model.metabolites.gmagg_e.formula = 'C39H64N8O19'

In [317]:
model.reactions.UAAAGGLAAT.add_metabolites({model.metabolites.h_c:-1})

In [318]:
#here add a phosphate donor group
model.reactions.ACMUMPT.add_metabolites({model.metabolites.pi_c:-1, model.metabolites.h2o_c:1.0})

In [319]:
model.metabolites.acmum6p_c.formula = 'C11H17NO11P'

In [320]:
model.metabolites.acmum_c.formula = 'C11H18NO8'
model.metabolites.acmum_e.formula = 'C11H18NO8'

In [321]:
model.metabolites.acmum_c.charge = -1

In [322]:
model.metabolites.acmum6p_c.charge = -3

For rct BTN5AMPL, I will get rid of the enzyme that catalyses the reaction here. So the reaction appears to have a free AMP bound to free biotin, which is a mimic of the true reactions. 
This is because the enzyme-bound biotin is currently not in the model, and I dont want to create new dead-end and orphan metabolites as these will not run unless they are fixed.

In [323]:
model.metabolites.btn_c.name = 'Biotin'

In [324]:
model.reactions.BTN5AMPL.add_metabolites({model.metabolites.btn_c:1.0, model.metabolites.coa_c:1.0, model.metabolites.ACP_c:-1, model.metabolites.h2o_c:-1, model.metabolites.h_c:2})

In [325]:
model.metabolites.b5amp_c.formula = 'C20H27N7O9PS'
model.metabolites.b5amp_c.charge = -1

In [326]:
model.metabolites.btn_c.formula = 'C10H15N2O3S'
model.metabolites.btn_c.charge = -1

For rcts ACCOAACT and ACCOATT we need to add an extra metabolite: 1-Acyl-sn-glycerol-3-phosphate. 

In [327]:
model.add_metabolites(Metabolite(id = 'aglyc3p_c'))

In [328]:
model.metabolites.aglyc3p_c.name = '1-Acyl-sn-glycerol 3-phosphate'
model.metabolites.aglyc3p_c.formula = 'C4H7O7PR'
model.metabolites.aglyc3p_c.charge = -2
model.metabolites.aglyc3p_c.compartment = 'c'
model.metabolites.aglyc3p_c.notes['KEGG'] = 'C00681'

In [329]:
model.reactions.ACCOAACT.add_metabolites({model.metabolites.aglyc3p_c:1.0})

In [330]:
model.metabolites.acoa_c.charge = -4
model.metabolites.acoa_c.formula = 'C22H32N7O17P3SR'

In [331]:
model.reactions.ACCOATT.add_metabolites({model.metabolites.aglyc3p_c:-1})

In [346]:
model.metabolites.pa_EC_c.formula = 'C5H7O8PR2'

The ACEDIA reaction has an undefined electron 

In [355]:
model.reactions.ACEDIA.add_metabolites({model.metabolites.hacc_c:2})

In [369]:
#Save&commit
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

In [5]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

In [6]:
#SELMELIG: this is a dead-end reaction anyway, so can be removed
rct = model.reactions.SELMELIG

In [8]:
model.remove_reactions(rct)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [9]:
#save&commit
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')